En este archivo, voy a desarrollar el segundo sistema de recomendación que se me pide para este proyecto.
Este sistema de recomendación funciona de la siguiente manera:
![alt text](../img/user_item.jpg "Title")

Suponiendo que tenemos a un usuario 1 y a un usuario 2. Al usuario 1 le gusta el cubo de rubik, los dados y un juego de cartas. Al usuario 2 también le gusta el cubo de rubik y los dados, pero no ha jugado al juego de cartas.
 De forma muy simple, el sistema de recomendación user-item reconocería que estos dos usuarios comparten gustos similares, y al usuario 2 se le recomendaría el juego de cartas, que es jugado por otro usuario con gustos similares al suyo.  